# Sharpe optimal portfolio - comparisons

This example compares the performance of several risk-Sharpe optimal portfolios. 

The CVaR, SMCR, MAD, LSSD, BTAD, BTSD, GINI, SD and MV risk-based portfolios are considered. We use the `rtype=Sharpe` optimization flag with a risk-free rate of 0 (default value).

We start by importing **azapy** and other useful packages.

In [65]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.__version__}")

'0.1.1'

### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `force=True` get historical market data directly from the provider (in this case yahoo)
- `save=False` the historical market data is not saved

>For more information see https://azapy.readthedocs.io/en/latest/.

In [66]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = "2012-01-01"
edate = "today"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, force=True, save=False)

get GLD from yahoo only
get TLT from yahoo only
get XLV from yahoo only
get VGT from yahoo only
get PSJ from yahoo only

Request between 2012-01-01 : 2022-06-15
                    GLD         TLT         XLV         VGT         PSJ
source            yahoo       yahoo       yahoo       yahoo       yahoo
force              True        True        True        True        True
save              False       False       False       False       False
file_dir         outDir      outDir      outDir      outDir      outDir
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2631        2631        2631        2631        2631
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2022-06-15  2022-06-15  2022-06-15

### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio,
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR` equal weighted,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR` equal weighted,
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSSD'` is the mLSSD optimal portfolio with weights `coef_LSSD`,
- `'P_Omega'` is the mBTAD optimal portfolio with threshold levels `alpha_BTAD` equal weighted,
- `'P_Sortino'` is the mBTSD optimal portfolio with threshold levels `alpha_BTSD` equal weighted,
- `'P_GINI'` is the Gini optimal portfolio,
- `'P_MV'` is the MV optimal portfolio,
- `'P_SD'` is the SD optimal portfolio.


- In all cases the optimization type is set to `rtype='Sharpe'` (optimal portfolio with the same risk profile as the equal weighted portfolio) with risk-free rate mu0=0.
- The `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` where it is set to 1.25 years (for computational speed convenience).

> Note:
> - mBTAD-Sharpe ratio is called Omega ratio 
> - mBTSD-Sharpe ratio is called Sortino ratio 
> - MV-Sharpe ratio is different than (SD) Sharp ratio 

In [67]:
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
rtype = 'Sharpe'
coef_MAD = [1./3.] * 3
coef_LSSD = [1./3.] * 3
alpha_BTAD = [-0.01, 0]
alpha_BTSD = [-0.01, 0]

models = {'P_CVaR-Sharpe': {'type': 'Port_CVaR', 'm_param': {'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': 3.25}},
          'P_SMCR-Sharpe': {'type': 'Port_SMCR', 'm_param': {'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': 3.25}},
          'P_MAD-Sharpe': {'type': 'Port_MAD', 'm_param': {'coef': coef_MAD, 'rtype': rtype, 'hlength': 3.25}},
          'P_LSSD-Sharpe': {'type': 'Port_LSSD', 'm_param': {'coef': coef_LSSD, 'rtype': rtype, 'hlength': 3.25}},
          'P_Omega': {'type': 'Port_BTAD', 'm_param': {'alpha': alpha_BTAD, 'rtype': rtype, 'hlength': 3.25}},
          'P_Sortino': {'type': 'Port_BTSD', 'm_param': {'alpha': alpha_BTSD, 'rtype': rtype, 'hlength': 3.25}},
          'P_GINI-Sharpe': {'type': 'Port_GINI', 'm_param': {'rtype': rtype, 'hlength': 1.25}},
          'P_MV-Sharpe': {'type': 'Port_MV', 'm_param': {'rtype': rtype, 'hlength': 3.25}},
          'P_Sharpe': {'type': 'Port_SD', 'm_param': {'rtype': rtype, 'hlength': 3.25}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [68]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [69]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [70]:
_ = ps.port_view_all(componly=True, fancy=True)

### Portfolio performances

In [71]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_N,9.35,-20.67,0.452437,2022-06-14,2021-11-09,NaN
P_GINI-Sharpe,8.59,-22.02,0.390269,2018-12-24,2018-09-14,2019-12-26
P_MV-Sharpe,7.17,-18.70,0.383431,2022-06-14,2021-12-30,NaN
P_Omega,7.00,-18.39,0.380615,2020-03-20,2020-02-19,2020-07-15
P_Sortino,6.68,-18.31,0.364640,2020-03-20,2020-02-19,2020-06-23
P_CVaR-Sharpe,6.48,-17.80,0.363844,2020-03-20,2020-02-19,2020-06-22
P_LSSD-Sharpe,6.51,-19.02,0.342497,2020-03-20,2020-02-19,2020-07-20
P_SMCR-Sharpe,6.47,-18.99,0.340745,2020-03-20,2020-02-19,2020-07-17
P_MAD-Sharpe,7.22,-22.60,0.319305,2020-03-20,2020-02-19,2020-07-27


### Annual returns

Note the best and worst performer each year.

> The flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.

In [72]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_CVaR-Sharpe,P_GINI-Sharpe,P_LSSD-Sharpe,P_MAD-Sharpe,P_MV-Sharpe,P_N,P_Omega,P_SMCR-Sharpe,P_Sharpe,P_Sortino
year,,,,,,,,,,
2015,-2.59%,-3.28%,-2.66%,-1.38%,-1.38%,-1.59%,-3.89%,-4.70%,-1.71%,-4.56%
2016,1.31%,-1.80%,1.32%,0.66%,2.92%,8.21%,1.06%,1.49%,1.91%,1.34%
2017,25.79%,37.17%,25.84%,28.05%,25.63%,23.12%,27.54%,25.14%,26.65%,26.01%
2018,0.35%,-2.94%,0.46%,2.37%,1.17%,2.52%,1.01%,0.08%,3.05%,0.59%
2019,24.56%,22.37%,25.09%,29.20%,26.39%,25.77%,27.69%,26.28%,27.87%,26.27%
2020,12.98%,21.47%,11.84%,11.65%,10.40%,25.54%,11.48%,13.06%,8.97%,12.83%
2021,3.29%,12.55%,4.92%,7.87%,8.58%,6.98%,4.43%,5.54%,8.33%,4.05%
2022,-14.99%,-15.20%,-15.73%,-19.88%,-16.30%,-17.73%,-14.59%,-15.76%,-20.20%,-14.34%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [73]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_Omega portfolio

In [74]:
pp['P_Omega'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021,2022
month,,,,,,,,
1,nan%,-2.08%,2.36%,4.03%,6.41%,1.62%,-1.89%,-4.35%
2,nan%,1.00%,3.49%,-0.06%,4.60%,-3.62%,-3.91%,-0.07%
3,nan%,1.95%,0.85%,-0.05%,-0.55%,-6.39%,-2.04%,1.11%
4,nan%,-0.99%,1.96%,1.30%,3.46%,8.89%,2.33%,-5.68%
5,nan%,2.43%,3.25%,4.99%,-4.15%,3.31%,2.62%,-2.07%
6,-2.03%,2.50%,-2.07%,-4.11%,8.11%,-0.91%,-1.92%,-3.79%
7,2.16%,3.75%,3.35%,0.57%,3.78%,7.33%,2.55%,nan%
8,-7.43%,-0.45%,2.60%,6.79%,-2.84%,-0.42%,0.70%,nan%
9,-2.61%,0.27%,1.83%,-0.80%,-0.30%,-3.28%,-4.10%,nan%


### P_Omega performance

An example of individual portfolio performance inquiry. 

In [75]:
pp['P_Omega'].port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_Omega,7.00,-18.39,0.380615,2020-03-20,2020-02-19,2020-07-15
VGT,18.52,-31.84,0.581743,2020-03-23,2020-02-19,2020-06-09
XLV,14.50,-28.40,0.510377,2020-03-23,2020-01-22,2020-07-15
PSJ,14.59,-50.02,0.291617,2022-06-13,2021-02-12,NaN
TLT,1.65,-34.76,0.047501,2022-06-14,2020-08-04,NaN
GLD,0.88,-42.11,0.020782,2015-12-17,2012-10-04,2020-07-22


### P_Omega drawdowns

In [76]:
pp['P_Omega'].port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-18.39,2020-03-20,2020-02-19,2020-07-15
2,-16.76,2022-06-14,2021-11-17,NaN
3,-16.40,2018-12-24,2018-08-31,2019-03-13
4,-14.16,2015-09-28,2015-08-05,2016-07-01
5,-9.89,2021-03-08,2020-08-06,2021-07-29


### P_Omega weights

In [77]:
pp['P_Omega'].get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,0.00,0.00,7.21,0.00,92.79
1,2015-09-25,2015-09-24,2012-06-22,0.00,12.32,22.49,0.00,65.19
2,2015-12-28,2015-12-24,2012-09-24,0.00,26.31,30.06,22.88,20.75
3,2016-03-28,2016-03-24,2012-12-24,0.00,0.00,38.21,23.82,37.97
4,2016-06-27,2016-06-24,2013-03-22,0.00,9.72,39.56,22.77,27.95
5,2016-09-27,2016-09-26,2013-06-26,0.00,28.11,50.41,1.33,20.15
6,2016-12-27,2016-12-23,2013-09-23,0.00,0.00,36.08,63.92,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.00,0.00,32.62,67.38,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.00,0.00,30.58,69.42,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.00,4.15,29.86,65.99,0.00
